In [69]:
#define function to read mdEditor files, and write out as csv 
#and shapefile with geography, if applicable
#by Tamatha A. Patterson; verson 5; December 2022
#distribution section updated.
#extent merged with metadata and written to shapefile. #FeatureCollection type handled.
#associations section updated.

def mdEditor_read(metadataToRead, contact_md, csvname, workspace, recordtype = 'all'):
    #recordtype used to write chosen resource types  \"all\", \"project\", etc.in future versions
    #extent/geography read when is True, skipped when False.
    import os
    import json
    import csv
    import pandas as pd
    import geopandas as gpd
    import collections
    import fiona
    from shapely.geometry import Point, LineString, Polygon
    
    def removeComma(string): #define remove comma function
        return string.replace(",","; ")
    
    def listToString(s): #define converting list to a string format
        str1 = "; "
        return (str1.join(s))
    
    def def_value():
        return "none"
    
    os.chdir(workspace) # assign working directory
    
    df = pd.read_json(metadataToRead)#read JSON metadata files into dataframe
    values = df.get('data') #assigns metadata to values
    for e in range(0, len(values)): #json file may have multiple metadata records
        element = values[e] #assign list value e containing the metadata #keys= id, attributes, type
        ID = element.get('id') #get metadata id
        attribute = element.get('attributes') # keys= profile, json, data-updated
        typpe = element.get('type') #get metadata type
        if typpe != 'records':  
            # skip if metadata is not a record and is a data dictionary, setting, schemas, custom-profiles...
            continue  #go to next record

        #parse 'attribute' to create profile, json, and date-updated\n",
        dateUpdate = attribute.get('date-updated') #create dateUpdate value--where does this date come from???
        jsondata = attribute.get('json') #create'json' data value
        profile = attribute.get('profile') #create 'profile' value
        
        #convert string to dictionary
        jsondatadict = json.loads(jsondata)  #3 keys = schema, metadata, mdDictionary
        schema = jsondatadict.get('schema')
        metadata = jsondatadict.get('metadata') #4 keys = metadataInfo, resourceInfo, associatedResource, resourceDistribution\n",
        mdDictionary = jsondatadict.get('mdDictionary')
        #get metadata key entries
        metadataInfo = metadata.get('metadataInfo') #6 Keys = metadataIdentifier, metadataContact, defaultMetadataLocale, metadataDate, parentMetadata, metadataStatus
        resourceInfo = metadata.get('resourceInfo') #12 keys = resourceType, citation, pointOfContact, abstract, shortAbstract, status, defaultResourceLocale, extent, keyword, purpose, taxonomy, timePeriod
        associatedResource = metadata.get('associatedResource') #is list
        resourceDistribution = metadata.get('resourceDistribution') #keys = n

        #parse metadataInfo dictionary 6 keys
        metadataIdentifier = metadataInfo.get('metadataIdentifier') #Harvest as ID to fields
        metadataContact = metadataInfo.get('metadataContact')
        defaultMetadataLocale = metadataInfo.get('defaultMetadataLocale')
        metadataDate = metadataInfo.get('metadataDate')
        parentMetadata = metadataInfo.get('parentMetadata')
        metadataStatus = metadataInfo.get('metadataStatus') #Harvest as status to fields

        #get metadata uuid identifier; autocreated in mdEditor\n",
        if metadataIdentifier['namespace'] == 'urn:uuid':
                metaIdentifier = metadataIdentifier.get('identifier')

        #parse resourceInfo 12 keys: 'resourceType', 'citation', 'pointOfContact', 'abstract', 'shortAbstract', 'status', \n",
        #...'defaultResourceLocale', 'extent', 'keyword', 'purpose', 'taxonomy', 'timePeriod'\n",
        resourceType = resourceInfo.get('resourceType')
        citation = resourceInfo.get('citation')
        pointOfContact = resourceInfo.get('pointOfContact')
        abstract = removeComma(resourceInfo.get('abstract')) #harvested to fields
        if resourceInfo.get('shortAbstract') == None:
            shortAbstract = " "
        else:
            shortAbstract = removeComma(resourceInfo.get('shortAbstract'))#harvested as shortAbstract to fields
        statusList = resourceInfo.get('status')
        status = statusList[0]#harvest as status to fields
        defaultResourceLocale = resourceInfo.get('defaultResourceLocale')
        extent = resourceInfo.get('extent')
        keyword = resourceInfo.get('keyword')
        if resourceInfo.get('purpose') == None:
            purpose = " "
        else:
            purpose = removeComma(resourceInfo.get('purpose')) #harvested to fields
        taxonomy = resourceInfo.get('taxonomy')
        timePeriod = resourceInfo.get('timePeriod')

        #find last update date from metadataDate
        #Consider comparing this to last run date and only reading metadata updated after????
        if len(metadataDate) == 1:
            lastUpdate = metadataDate[0].get('date') 
            dateType = metadataDate[0].get('dateType')
        else:
            if len(metadataDate) > 1:
                for i in metadataDate:
                    if i.get('dateType') == "lastUpdate":
                        lastUpdate = (i.get('date')).split('T')[0]
                        dateType = 'last updated'
                    else: 
                        dateType = i.get('dateType')
                        lastUpdate = (i.get('date')).split('T')[0]                   

        #parse resource Type info
        typelist = resourceType[0]
        typee = typelist.get('type')
        typeename = typelist.get('name')
        if typeename != None: 
            typeename = removeComma(typelist.get('name'))

        #parse citation info
        title = removeComma(citation.get('title')) #harvested as title to fields
        dates = citation.get('date')
        responsibleParty = citation.get('responsibleParty')
        altTitle = citation.get('alternateTitle')
        if altTitle != None:
            altTitle = listToString(altTitle)#Harvested as altTitle to fields
            altTitle = removeComma(altTitle)
        
        #Get and format startDate and endDate
        try:
            startDate = (timePeriod.get('startDateTime','None')).split('T')[0]
            end = timePeriod.get('endDateTime', 'None')
            if end == None:
                endDate = 'onGoing'
            else:
                endDate = end.split('T')[0]
        except:
            startDate = 'None'
            endDate = 'None'
        
        #create empty list for keywords
        klist =[]
        #loop through keyword thesaurus and add keywords to keyword list
        try:
            for g in range(0, len(keyword)):
                word = keyword[g]
                word1 = word.get('keyword')
                for h in range(0, len(word1)):
                    word2 = word1[h]
                    word3 = word2.get('keyword')
                    klist.append(word3)
            keywords = listToString(klist)
        except:
            keywords = 'None'
            
        #parse species names from taxonomy; may need to loop if more than one species.
        try: # check for taxonomy entry
            taxdic = taxonomy[0]
            taxClass = taxdic.get('taxonomicClassification')
            taxSys = taxdic.get('taxonomicSystem')
            taxvoucher = taxdic.get('voucher')
            taxClass1 = taxClass[0]
            taxClass1.keys()
            taxSysID = taxClass1.get('taxonoicSystemID')
            taxLevel = taxClass1.get('taxonomicLevel')
            taxName = taxClass1.get('taxonomicName')
            taxSubClass = taxClass1.get('subClassification')
            taxIs = taxClass1.get('isITIS')
            taxSub0 = taxSubClass[0]
            taxSysID0 = taxSub0.get('taxonoicSystemID')
            taxLevel0 = taxSub0.get('taxonomicLevel')
            taxName0 = taxSub0.get('taxonomicName')
            taxSubClass0 = taxSub0.get('subClassification')
            taxIs0 = taxSub0.get('isITIS')
            subKingdom =taxSubClass0[0]
            infraKingdomL = subKingdom.get('subClassification')
            infraKingdom = infraKingdomL[0]
            phylumL = infraKingdom.get('subClassification')
            phylum = phylumL[0]
            subphylumL = phylum.get('subClassification')
            subphylum = subphylumL[0]
            infraphylumL = subphylum.get('subClassification')
            infraphylum = infraphylumL[0]
            superclassL = infraphylum.get('subClassification')
            superclass = superclassL[0]
            classL = superclass.get('subClassification')
            classD = classL[0]
            orderL = classD.get('subClassification')
            order = orderL[0]
            familyL = order.get('subClassification')
            family = familyL[0]
            genusL = family.get('subClassification')
            genus = genusL[0]
            taxname = genus.get('taxonomicName') #harvested to fields
            t =[taxname]
            comname = genus.get('commonName') #harvested to fields
            t.append(comname)
            comname = listToString(comname)
            comname = removeComma(comname)
        except:  # if no taxomony, then assign 'none'
            taxname = "none"
            comname = "none"  
    
    #Get Associated Resource Info
    #if assocated resources are selected from metadata records.... then:
        try:
            assoclist = []  # create empty list of assocated resource
            assocdic = associatedResource[0] #dict_keys(['resourceType', 'resourceCitation', 'associationType', 'initiativeType'])
            aresourceType = assocdic.get('resourceType')
            aresourceCitation = assocdic.get('resourceCitation')
            aassociationType = assocdic.get('associationType')#harvest as associationType ie parentProject
            ainitiativeType = assocdic.get('initiativeType') #initiativeType ie project
            if aassociationType == 'parentProject' and aresourceType != None:  #this is product metadata record
                aresourceType1 = aresourceType[0] #i.e.{'type': 'project', 'name': 'Lesser Yellowlegs Ecology'}
                atype = aresourceType1.get('type') #get first entry to remove brackets
                aname = aresourceType1.get('name')
                addassoc = "parentProject: " + aname
                assoclist.append(addassoc)
            elif aassociationType == 'product': #this is project metadata record
                for a in range (0, len(associatedResource)):
                    l = associatedResource[a]
                    k = l.get('mdRecordId')
                    assoclist.append(k)
            else:
                assoclist = 'no assocated records'
        except:
            assoclist = 'no assocated records present'

        #print ('associations are: ', assoclist)

    
    #get resourceDistribution metada
        resourceDistribution = metadata.get('resourceDistribution')
        distlist = {} #create empty distribution list
        try:
            for d in range(0, len(resourceDistribution)): #iterate through resourceDistribution info list
                distributor = resourceDistribution[d]
                dist = dict(distributor)
                dist0 = dist.get("distributor")
                dist1 = dist0[0] #dictionary keys = 'contact', 'transferOption'
                contact = dist1.get('contact')
                order = dist1.get('orderProcess')
                transopt = dist1.get('transferOption')
                distrole =contact.get('role') #harvested as distributor role to distlist
                distparty = contact.get('party') #distrbutor contact identifiers
                #if len(distparty) > 1:
                    #for org in range(0,len(distparty)): 
                        #distID = distparty[0]
                        #distributorID = distID.get('contactId') #harvest distributor ID & compare with contact master list be
                #else:
                distID = distparty[0]
                distributorID = distID.get('contactId') #harvest distributor ID & compare with contact master list be
                transopt1 = transopt[0]
                transopt2 = transopt1.get('onlineOption')
                transopt3 = transopt2[0]
                onlineName = transopt3.get('name') #harvested to distlist
                onlineUri = transopt3.get('uri') #harvested to distlist
                distInfo =[distrole, distributorID, onlineName, onlineUri]
                distlist[d] = distInfo
                #print ("distlist = ", distlist)
                distInfoString = '; '.join(distInfo)
            
        except:
            print(title, ' has NO distribution metadata')
            distInfoString = ' '
      
    #POINTS of CONTACT
        #read Master Contact JSON metadata file into dataframe
        contactmetadata = pd.read_json(contact_md)
        contactmd1 = dict(contactmetadata)
        contactmd2 = contactmd1.get('data')
        
        POC = collections.defaultdict(list) # create empty dictionary for contacts
        POCvalues = []
        count = 0

        #iterate through master contact metadata
        for k in contactmd2:
            contactmd3 = contactmd2[count]
            contactmd4 = contactmd3.get('attributes')
            contactmd5 = contactmd4.get('json')
            contactmd6 = json.loads(contactmd5)
            contactmd7 = dict(contactmd6)
            contactIDmd = contactmd7.get('contactId') #harvest id#
            count += 1

            for id in distlist:
                if distlist[id][1] == contactIDmd:
                    contactisOrganizationmd = contactmd7.get('isOrganization')
                    contactName = contactmd7.get('name') #havest as distributor name to fields
                    contactMemberOf = contactmd7.get('memberOfOrganization')
                    contactemail = contactmd7.get('electronicMailAdddress')
                    contactType = contactmd7.get('contactType')
                    distlist[id][1] = contactName 

            # iterate through contacts from metadata
            for j in pointOfContact:
                party = j.get('party')
                for p in range(0, len(party)):
                    partyContact = party[p]
                    partyContactID = partyContact.get('contactId') #id to compare in master contact list
                    role = j.get('role')

                    #compare master list contact ID with metadata contact ID
                    if contactIDmd == partyContactID:
                        contactisOrganizationmd = contactmd7.get('isOrganization')
                        contactName = contactmd7.get('name')
                        contactMemberOf = contactmd7.get('memberOfOrganization')
                        contactemail = contactmd7.get('electronicMailAdddress')
                        contactType = contactmd7.get('contactType')
                        POC[role].append(contactName)
                    #else:
                        continue

        owner = listToString(POC['owner'])
        PointOC = listToString(POC['pointOfContact'])
        princ = listToString(POC['principalInvestigator'])
        custodian = listToString(POC['custodian'])
        admin = listToString(POC['administrator'])
        originator = listToString(POC['originator'])
        contributor = listToString(POC['contributor'])
        #distlistString = '; '.join(distInfo) 

   #Write vales to CSV
        fields = [ID, typee, title, altTitle, typeename, purpose, abstract, shortAbstract, 
                  PointOC, owner, princ, custodian, admin, originator, contributor, startDate, endDate, lastUpdate, status, 
                  metaIdentifier, metadataStatus, keywords, taxname, comname, distInfoString, assoclist] #gEid?

        #write files to csv.
        with open (csvname, 'a', newline = '') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(fields)     
        
    return
    print("ALL DONE! CSV created./n","Number of contacts in master list = ", len(contactmd2))

In [70]:
#testing for individual metadata files:

#metadataToRead = 'C:\\Users\\tpatterson\\OneDrive - DOI\\Documents\\DM_Metadatafiles\\CatalogCSV\\PacificFlywayWinterBrantSurvey-mdeditor-20220705-150787.json'
#metadataToRead = 'C:\\Users\\tpatterson\\OneDrive - DOI\\Documents\\DM_Metadatafiles\\CatalogCSV\\EielsonAvianStudy-mdeditor-20220627-170675.json'
#metadataToRead = 'C:\\Users\\tpatterson\\OneDrive - DOI\\Documents\\DM_Metadatafiles\\CatalogCSV\\SeaDuckAtlas-mdeditor-20220629-110616.json'
#metadataToRead = 'C:\\Users\\tpatterson\\OneDrive - DOI\\Documents\\DM_Metadatafiles\\CatalogCSV\\LesserYellowlegEcology-mdeditor-20220705-210739.json'
metadataToRead = 'C:\\Users\\tpatterson\\OneDrive - DOI\\Documents\\DM_Metadatafiles\\CatalogCSV\\mbmwa_013_Izembek_Fall_Brant_Survey-init-mdeditor-20221110.json'
#metadataToRead = 'C:\\Users\\tpatterson\\OneDrive - DOI\\Documents\\DM_Metadatafiles\\CatalogCSV\\RedKnotStudy-mdeditor-20220627-160603.json'

# Pathway to the contacts file you want to use to check against existing vs. new contacts; i.e., master AK contacts file\n",
contact_md = 'C:\\\\Users\\\\tpatterson\\\\OneDrive - DOI\\\\Documents\\\\DM_Metadatafiles\\\\AKContactsmdeditor-20211228-101265.json'

# Pathway to csv file where to write metaata
csvname = 'C:\\\\Users\\\\tpatterson\\\\OneDrive - DOI\\\\Documents\\\\DM_Metadatafiles\\\\CatalogCSV\\\\catalogCSVvtest.csv'

workspace = 'C:\\Users\\tpatterson\\OneDrive - DOI\\Documents\\DM_Metadatafiles\\CatalogCSV'

mdEditor_read(metadataToRead, contact_md, csvname, workspace)

Alasaka Izembek Fall Brant Aerial Survey 1976-Present  has NO distribution metadata
this is a feature collection extent
Izembek National Wildlife Refuge, simplified boundary.


ValueError: Unknown column geometry

In [37]:
#Search Migratory Birds Management RDR folder for mdeditor files to extract metadata
#Count number of preserved mdEditor records
import os
RDR = '\\\\ifw7ro-file.fws.doi.net\\datamgt\\mbm'
MBMmetadataNo = 0
program = "Migratory Bird Manangement"

# Pathway to the contacts file you want to use to check against existing vs. new contacts; i.e., master AK contacts file\n",
contact_md = 'C:\\\\Users\\\\tpatterson\\\\OneDrive - DOI\\\\Documents\\\\DM_Metadatafiles\\\\AKContactsmdeditor-20211228-101265.json'

# Pathway to csv file where to write metaata
csvname = 'C:\\\\Users\\\\tpatterson\\\\OneDrive - DOI\\\\Documents\\\\DM_Metadatafiles\\\\CatalogCSV\\\\catalogCSVPhase2v1.csv'

workspace = 'C:\\Users\\tpatterson\\OneDrive - DOI\\Documents\\DM_Metadatafiles\\CatalogCSV\\MBMExtentTest'

#loop through RDR folder structure and find mdeditor json files that is NOT in incoming folder
for root, dirs, files in os.walk(RDR,topdown=True):
    #print ("root=", root, "  dirs=", dirs, "  file=", files)
    for name in files:
        if 'incoming' not in root and 'mdeditor' in name and name.endswith('.json'):
            jfile = os.path.join(root,name)
            mdEditor_read(jfile, contact_md, csvname, workspace)
            MBMmetadataNo += 1
            print (jfile)
print ("Number of MBM completed mdeditor records in RDR = ",MBMmetadataNo) 

C:\Users\tpatterson\AppData\Local\Temp\1\ipykernel_2668\3647770736.py:403: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shape = dfmerge.to_file(polyname, encoding = 'utf-8')#, driver = 'ESRI Shapefile', schema = {"geometry": "Polygon", "properties":{"id":"int"}})


\\ifw7ro-file.fws.doi.net\datamgt\mbm\mbmjv_001_SeaDuck_Key_Habitat_Atlas\metadata\SeaDuckAtlas-mdeditor-20220629-110616.json
Alaska Red Knot Study 2010-Present  has NO distribution metadata
Alaska Red Knot Banding Data 2010-Present  has NO distribution metadata
Alaska Red Knot Nest Data 2010-Present  has NO distribution metadata
Alaska Red Knot Brood Data 2010-Present  has NO distribution metadata
Alaska Red Knot Geolocator Raw Data 2006-Present  has NO distribution metadata
Alaska Red Knot Reports 2011-Present  has NO distribution metadata


C:\Users\tpatterson\AppData\Local\Temp\1\ipykernel_2668\3647770736.py:403: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shape = dfmerge.to_file(polyname, encoding = 'utf-8')#, driver = 'ESRI Shapefile', schema = {"geometry": "Polygon", "properties":{"id":"int"}})


Alaska Red knot Study Data Management Plan  has NO distribution metadata
Alaska Red Knot Resight Data 2010-Present  has NO distribution metadata
\\ifw7ro-file.fws.doi.net\datamgt\mbm\mbmlb_003_Red_Knot_Breeding\metadata\RedKnotStudy-mdeditor-20220627-160603.json
Alaska Eielson Air Force Base Avian Study 2019-Present  has NO distribution metadata


C:\Users\tpatterson\AppData\Local\Temp\1\ipykernel_2668\3647770736.py:403: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shape = dfmerge.to_file(polyname, encoding = 'utf-8')#, driver = 'ESRI Shapefile', schema = {"geometry": "Polygon", "properties":{"id":"int"}})


...

C:\Users\tpatterson\AppData\Local\Temp\1\ipykernel_2668\3647770736.py:403: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shape = dfmerge.to_file(polyname, encoding = 'utf-8')#, driver = 'ESRI Shapefile', schema = {"geometry": "Polygon", "properties":{"id":"int"}})


\\ifw7ro-file.fws.doi.net\datamgt\mbm\mbmwa_010_Izembek_Winter_Brant_Aerial_Survey\metadata\PacificFlywayWinterBrantSurvey-mdeditor-20220705-150787.json


C:\Users\tpatterson\AppData\Local\Temp\1\ipykernel_2668\3647770736.py:403: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shape = dfmerge.to_file(polyname, encoding = 'utf-8')#, driver = 'ESRI Shapefile', schema = {"geometry": "Polygon", "properties":{"id":"int"}})


\\ifw7ro-file.fws.doi.net\datamgt\mbm\mbmwa_011_AK_Swan_Database\metadata\SwanDatabase-mdeditor-20220627-140673.json
Alasaka Izembek Fall Brant Aerial Survey 1976-Present  has NO distribution metadata


ValueError: A LinearRing must have at least 3 coordinate tuples

In [ ]:
#Search FES RDR folder for mdeditor files to extract metadata
#Count number of preserved mdEditor records
import os
RDR = '\\\\ifw7ro-file.fws.doi.net\\datamgt\\fes'
MBMmetadataNo = 0
program = "Fisheries & Ecological Services"

# Pathway to the contacts file you want to use to check against existing vs. new contacts; i.e., master AK contacts file\n",
contact_md = 'C:\\\\Users\\\\tpatterson\\\\OneDrive - DOI\\\\Documents\\\\DM_Metadatafiles\\\\AKContactsmdeditor-20211228-101265.json'

# Pathway to csv file where to write metaata
csvname = 'C:\\\\Users\\\\tpatterson\\\\OneDrive - DOI\\\\Documents\\\\DM_Metadatafiles\\\\CatalogCSV\\\\catalogCSVv3.csv'

#loop through RDR folder structure and find mdeditor json files that is NOT in incoming folder
for root, dirs, files in os.walk(RDR,topdown=True):
    #print ("root=", root, "  dirs=", dirs, "  file=", files)
    for name in files:
        if 'incoming' not in root and 'mdeditor' in name and name.endswith('.json'):
            jfile = os.path.join(root,name)
            mdEditor_read(jfile, contact_md, csvname)
            MBMmetadataNo += 1
            print (jfile)
print ("Number of FES completed mdeditor records in RDR = ",MBMmetadataNo) 